## Filter features
Now that we have the features in a dataframe we can filter for certain criteria to find the most in demand jobs for a certain job profile. The varibales to filter on will be job title, experience and degree.   
Initially pct to total post were used to give an accurate percantage of the job posts asking for certain skills. 
Later when developing treemaps for the dashboard. Evaluating percantage related to total job posts often didn't show information appriately. Showing groups with more general terms much larger. Due to this the percentage for each keyword is taken in relation to the total count of the the keyword group to which it belongs. Showing each keyword group as equal size and keywords portion of the group shown clearly. 
This Notebook we will:
- selecet 2 job profile as filter varibles 
- filter df and add keyword groups 
- compare skills from the two job profiles 

In [11]:
import numpy as np
import pandas as pd  
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import popular_data_skills.utils.keywords as kws
import popular_data_skills.config.config as config

In [12]:
job_data = pd.read_csv(config.FINAL_DATA_FILE)

keyword_groups = kws.Keywords()
keyword_groups.read_file(config.KEYWORD_GROUPS)

## Select 2 job profile as filter varibles 

In [20]:
# Varibles 
# Profile 1
job = 'analyst'
experience = '5+'
degree = 'phd'

# Profile 2
job2 = 'scientist'
experience2 = '5+'
degree2 = 'phd'

comparision_group = 'Skills'

## filter df and add keyword groups 

Next we will filter the df by the variables from one profile then sum the features to find how many times each feature is mentioned for each job profile. 
Next we can add keywords groups to combine related keywords.

In [14]:
### Filter df for Treemap plot 
# Helper functions for filtering info giving in profiles
def degree_level(degree :str):
    '''Filters info to provide list of values to filter the degree column'''
    if degree == 'phd':
        return ['phd', 'ms', 'bs','not_specified']
    elif degree == 'ms':
        return ['ms', 'bs','not_specified']
    elif degree == 'bs':
        return ['bs','not_specified']
    elif degree == 'none':
        return ['not_specified']
    elif degree == 'any':
        return ['phd', 'ms', 'bs','not_specified']

def experience_level(experience :str):
    '''Filters info to provide list of values to filter the experience column'''
    if experience == '5+':
        return ['0', '1', '2', '3-5', '5+']
    elif experience == '3-5':
        return ['0', '1', '2', '3-5']
    elif experience == '2':
        return ['0', '1', '2']
    elif experience == '1':
        return ['0', '1']
    elif experience == '0':
        return ['0']
    
    
# Drop unuseful columns 
info_columns = ['company_name', 'country', 'job_title', 'location', 'date_posted','applicant_count', 'job_description_lines' ]
job_data = job_data.drop(info_columns, axis=1)

# Filter data as per profile
def filter_df(df,job,experience,degree):
    '''Uses Helper functions experience level and degree_level to filter df according to profile varibles'''
    df = df[
    (df['job'] == job) & 
    (df['experience'].isin(experience_level(experience))) &
    (df['degree_level'].isin(degree_level(degree)))
    ]
    return df

filtered_df = filter_df(job_data,job,experience,degree)

# Drop catagorical columns use to filter df that are no longer needed
drop_columns =['job', 'experience', 'degree_level'] 
filtered_df = filtered_df.drop(columns=drop_columns)

filtered_df.head()

,a/b,advertising,architecture,automation,aws,azure,bash,bayesian,c,c#,...,masters degree,bachelor degree,power bi,problem solving,decision trees,random forest,structured data,relational data,unstructured data,unsupervised learning
367,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
379,1,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,0,0
382,0,1,0,1,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
385,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
389,0,0,0,1,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0


In [15]:
# Sum columns and sort into a summary df.
filtered_df = (filtered_df.sum().sort_values(ascending=False).astype(int))

filtered_df = (filtered_df.to_frame().reset_index())
filtered_df

,index,0
0,analytics,45
1,masters degree,44
2,degree,28
3,healthcare,24
4,sql,24
...,...,...
76,AI,0
77,economic,0
78,regression,0
79,supervised,0


### Get percentage of post as related the total post in keyword parent group 

In [16]:
def add_groups(row):
    ''' Adds groups taken from imported keywords list class. Used to add new column to df to group skills together'''
    for group, list in keyword_groups.dict.items():
        if row in list: 
            return group

filtered_df['groups'] = filtered_df['index'].apply(add_groups).sort_values()
#filtered_df['index'] = filtered_df['index'].replace({'no_degree' : 'no degree stated','degree' : 'any degree'})


def get_group_totals(df):
    '''Create two groupby dfs from filtered info. First df shows totals count for each group. Second collates total count of each skill'''
    group_total_df = df.groupby(by='groups',as_index=False).sum()
    group_total_df.rename(columns={0:'group_total'}, inplace = True)

    grouped_df = df.groupby(['groups', 'index'],as_index=False).sum()
    grouped_df.rename(columns={0:'count'}, inplace = True)
    return group_total_df, grouped_df

group_total_df, filtered_df = get_group_totals(filtered_df)


def get_ratio(row):
    ''' Uses dfs from get_group_totals to calulate percentage of each skill relative to its parent group '''
    mask = group_total_df[group_total_df['groups'] == row['groups']]
    group_total = mask['group_total'].values[0].astype(int)
    # #return the percantage of group
    return ((row['count'] / group_total) *100).astype(int) 

filtered_df['ratio']  = filtered_df.apply(lambda x: get_ratio(x), axis=1)

filtered_df

,groups,index,count,ratio
0,Business_area,advertising,2,2
1,Business_area,crm,1,1
2,Business_area,economic,0,0
3,Business_area,finance,13,18
4,Business_area,government,1,1
...,...,...,...,...
75,Topic,preparation,5,3
76,Topic,problem solving,16,9
77,Topic,statistics,19,11
78,Topic,testing,4,2


## Plot charts 

### Treeplot

In [17]:
def tree_map(job: str, experience: str, degree: str, comparision_group: str):
    ''' Return tree map of data filtered by parameters giving. '''
    # Create treemap based on filtered dfs
    fig = px.treemap(filtered_df, path=[px.Constant("all"),'groups','index'], values='ratio', labels= "images",)

    fig.update_traces(root_color="lightgrey")
    fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
    fig.data[0]['textfont']['size'] = 30

    fig.update_traces(textposition="middle center", selector=dict(type='treemap'))
    fig.data[0].texttemplate = "%{label}<br>%{value}%"

    return fig.show()

tree_map(job,experience,degree, comparision_group)

#### Filter df for job profile 2

In [26]:
### Filter dfs for comparision plots 

# filtered_df2 is filtered df based on profile two varibles 
filtered_df2 = filter_df(job_data,job2,experience2,degree2)
# Drop catagorical columns once used for filter 
drop_columns =['job', 'experience', 'degree_level'] 
filtered_df2 = filtered_df2.drop(columns=drop_columns)

# Wrangle df total counts of each skill sorted 
filtered_df2 = (filtered_df2
.sum()
.sort_values(ascending=False).astype(int)
)
filtered_df2 = (filtered_df2
.to_frame()
.reset_index()
)

# add groups using helper function
filtered_df2['groups'] = filtered_df2['index'].apply(add_groups).sort_values()
# filtered_df2['index'] = filtered_df2['index'].replace({'no_degree' : 'no degree stated',
#                                                      'degree' : 'any degree'})

# Create two dfs one for group totals and second for skills totals
group_total_df2, filtered_df2 = get_group_totals(filtered_df2)

filtered_df2['ratio']  = filtered_df2.apply(lambda x: get_ratio(x), axis=1)

#Creates three seperate dfs and x and y axes are create to compare in plots
# Dfs are filtered based on profile varibles and comparision group
# Bar1 is filtered based on profile 1 and comparision group. Top 10 in demand skills are given. 
bar1_filtered_df = filtered_df[filtered_df['groups'] == comparision_group].sort_values(by='ratio', ascending=True).tail(10)
y = bar1_filtered_df['ratio']
x = bar1_filtered_df['index'].str.capitalize()


# TODO fix y 
# Bar2 is filtered based on profile 2 and comparision group. This uses the most in demand skills from profile 1 for direct skill by skill comparision.
bar2_filtered_df = filtered_df2[filtered_df2['groups'] == comparision_group]
x_2 = x

y_list = []
for item in bar1_filtered_df['index']:
    new_y = bar2_filtered_df[bar2_filtered_df['index'] == item]['ratio'].values[0]
    y_list.append(new_y)
y_2 = y_list

# Bar3 is filtered based on profile 2 and comparision group. It sorts the top 10 indemand skills for profile 2 
bar3_filtered_df = filtered_df2[filtered_df2['groups'] == comparision_group].sort_values(by='ratio', ascending=True).tail(10)
x_3 = bar3_filtered_df['index'].str.capitalize()
y_3 =  bar3_filtered_df['ratio']

print(bar1_filtered_df)
print(bar2_filtered_df)


    groups            index  count  ratio
35  Skills           deploy      3      4
30  Skills            cloud      3      4
36  Skills           mining      4      5
33  Skills   decision trees      4      5
26  Skills     architecture      4      5
39  Skills     optimization      4      5
27  Skills       automation      5      7
34  Skills             deep      6      8
45  Skills  relational data      7     10
50  Skills    visualization     15     22
    groups                  index  count  ratio
25  Skills                    a/b     13     19
26  Skills           architecture      8     11
27  Skills             automation     25     37
28  Skills               bayesian     10     14
29  Skills         classification     19     28
30  Skills                  cloud     27     40
31  Skills             clustering     16     23
32  Skills              dashboard      2      2
33  Skills         decision trees     39     58
34  Skills                   deep     43     64
35  Skills

### Comparision bar plot 

In [28]:
### Create grouped bar chart to compare top ten skills from profile one with same skills for profile two 

fig = go.Figure(data=[
    # Grouped bar principal plot
    go.Bar(
        name=f'''Job: {job}
        Experience: {experience}
        Degree: {degree}''', 
        x=y, 
        y=x,
        marker=dict(
            color='rgb(128, 0, 128)',
            ),
        text=y,
        textposition='inside', 
        orientation='h'
        ),
    # Grouped bar secondary plot
    go.Bar(
        name=f'''Job: {job2}
        Experience: {experience2}
        Degree: {degree2}''', 
        x=y_2, 
        y=x_2, 
        marker=dict(
            color='rgba(120, 140, 120, 0.5)',
            ),
        text= y_2,
        textposition='inside', 
        orientation='h'
        )
])

fig.update_layout(
    barmode='group', 
    height= len(y) * 75,
    )

# Title 
fig.update_layout(
    title=dict(
        text= "Profile Comparsion",
        y=0.92,
        x=0.15,
        xanchor='center',
        yanchor= 'top',
        font = dict(
        family="verdana",
        size=24,
        color='rgb(128, 0, 128)'
        ),
    ),

    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, .85],
    ),
    legend=dict(x=0.4, y=0.93, font_size=16, ),
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    )

# Annotations
annotations = []
# Foot note 
annotations.append(dict(xref='paper', yref='paper',
                        x=0.2, y=-0.109,
                        text='Figures are number of job posts a certain skills is mentioned in. As a percentage of total of the group',
                        font=dict(family='Arial', size=10, color='rgb(150,150,150)'),
                        showarrow=False))

# Title
annotations.append(dict(xref='paper', yref='paper',
                        x=0, 
                        y=0.98,
                        align = "left",
                        text=('Most in demand skills for job profile 1 (purple).<br>Compared with profile 2 (grey)</br>'),
                        font=dict(family='Arial', size=14, color='rgb(150,150,150)'),
                        showarrow=False))

fig.update_layout(annotations=annotations)


fig.show()

### Most popular skills from each job profile 

In [27]:
### Create two bar plots side by side to compare most in demand skills from each profile 

# Creating two subplots
fig = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                    shared_yaxes=False,
                    #vertical_spacing=0.0001
                    )

# Add bar chart for Profile 1 
fig.add_trace(go.Bar(
    x=y,
    y=x,
    marker=dict(
        color='rgba(50, 171, 96, 0.6)',
        line=dict(
            color='rgba(50, 171, 96, 1.0)',
            width=1),
    ),
    name= f'<b>{str.capitalize(job)}</b>  Exp: <b>{experience}</b>  Degree: <b>{str.capitalize(degree)}</b>        ',
    orientation='h',
), 1, 1)

# Add bar chart for Profile 2
fig.add_trace(go.Bar(
    x=y_3, 
    y=x_3,
    marker=dict(
        color='rgb(128, 0, 128)',
        line=dict(
            color='rgb(128, 0, 128)',
            width=1,)
    ),
    name=f'<b>{str.capitalize(job2)}</b>  Exp: <b>{experience2}</b>  Degree: <b>{str.capitalize(degree2)}</b>        ', 
    orientation='h',
), 1, 2)

# Title 
fig.update_layout(
    title=dict(text='Most in demand skills',
               font=dict(
                   size=26,
               ),
               x = 0.5,
               xanchor='center'
    ),
    
    # Axes       
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        side= 'right',
      
        ticklabelposition="inside",
   
        tickfont = dict(
            size=15,
           # color='rgba(102, 102, 102, 0.8)',
           # family=  "Droid Sans Mono",
            ),
        domain=[0.1, 0.85],
    ),
    yaxis2=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        ticklabelposition="inside",   
            tickfont = dict(
            size=15,
           # color='rgba(102, 102, 102, 0.8)',
           # family=  "Droid Sans Mono",
            ),
        linecolor='rgba(102, 102, 102, 0.8)',
        linewidth=2,
        domain=[0.1, 0.85],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=False,
        showgrid=True,
        autorange="reversed",
        domain=[0.1, 0.49],
    ),
    xaxis2=dict(
        zeroline=False,
        showline=False,
        showticklabels=False,
        showgrid=True,
        domain=[0.51, 0.9],
        #side='top',
        #dtick=25000,
    ),
    
    legend=dict(
        x=0.5,
        y=1.048, font_size=12, orientation = 'h', xanchor='center',),
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    #showlegend=False,
)


# Annotations 
annotations = []
# Adding percentage labels 
for yd, y_3d, xd, x_3d in zip(y, y_3, x, x_3):
    # Percentage labels for Profile 1 plot
    annotations.append(dict(xref='x1', yref='y1',
                            y=xd, x= 0.5,
                            text=str(yd) + '%',
                            font=dict(family='Arial', size=12,
                                      color='rgb(0, 0, 0)'),
                            showarrow=False))
    # Percentage labels for Profile 2 plot
    annotations.append(dict(xref='x2', yref='y2',
                            y=x_3d, x=0.5,
                            text=str(y_3d) + '%',
                            font=dict(family='Arial', size=12,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
# Footnotes
annotations.append(dict(xref='paper', yref='paper',
                        x=-0.2, y=-0.109,
                        text='Figures are number of job posts a certain skills is mentioned in. As a percentage of total of the group',
                        font=dict(family='Arial', size=10, color='rgb(150,150,150)'),
                        showarrow=False))

fig.update_layout(annotations=annotations)

fig.show() 